# Code for calculating the final dataframes for regression of the impact of the flow of momentum on managerial decision-making. 

In [ ]:
#find the maximum and minimum values for pc and pcv used for normalizing the momentum values
import os
from progressbar import ProgressBar
import pandas as pd
import pickle

#create list with all needed files
path = '/Users/gebruiker/Dropbox/Master Thesis/Master thesis RKC Waalwijk/Code'
pc_matches = []
for file in os.listdir(path):
    if 'pc_over_last_two_minutes_final' in file:
        pc_matches.append(file[30:37])

#create empty dataframe start iterating over each pitch control match
pbar = ProgressBar()
maximum = -1000
minimum = 1000

for game in pbar(pc_matches):
    #open pitch control file
    infile = open('pc_over_last_two_minutes_final' + game + '.p','rb')
    df_pc = pickle.load(infile)
    infile.close()
    #subtract pcv values so that both teams cannot have momentum at the same time
    df_pc['pcv home'] = df_pc['pcv home'] - df_pc['pcv away']
    df_pc['pcv away'] = df_pc['pcv away'] - df_pc['pcv home']
    #find minimum and maximum
    maximum_pcv = max(maximum, df_pc['pcv home'].max(), df_pc['pcv away'].max())
    minimum_pcv = min(minimum, df_pc['pcv home'].min(), df_pc['pcv away'].min())
    maximum_pc = max(maximum, df_pc['pc home'].max(), df_pc['pc away'].max())
    minimum_pc = min(minimum, df_pc['pc home'].min(), df_pc['pc away'].min())
    
    

In [ ]:
#Calculations that will be used for regression input for Momentum on decision making (PC and PCV calculations)
import os
from progressbar import ProgressBar
import pandas as pd
import pickle

#create list with all needed files
path = '/Users/gebruiker/Dropbox/Master Thesis/Master thesis RKC Waalwijk/Code'
pc_matches = []
for file in os.listdir(path):
    if 'pc_over_last_two_minutes_final' in file:
        pc_matches.append(file[30:37])

#create empty dataframe start looping over each pitch control match
final_df = pd.DataFrame()
pbar = ProgressBar()
for game in pbar(pc_matches):
    #open pitch control file
    infile = open('pc_over_last_two_minutes_final' + game + '.p','rb')
    df_pc = pickle.load(infile)
    infile.close()
    #subtract the pcv values so that both teams cannot have momentum at the same time and normalize the values for PC and PCV
    df_pc['momentum pcv home previous 2 minutes'] = df_pc['momentum pcv home previous 2 minutes'] - df_pc['momentum pcv away previous 2 minutes']
    df_pc['momentum pcv away previous 2 minutes'] = df_pc['momentum pcv away previous 2 minutes'] - df_pc['momentum pcv home previous 2 minutes']
    df_pc['pcv home'] = df_pc['pcv home'] - df_pc['pcv away']
    df_pc['pcv away'] = df_pc['pcv away'] - df_pc['pcv home']
    df_pc['momentum pcv home previous 2 minutes'] = (df_pc['momentum pcv home previous 2 minutes']-minimum_pcv)/(maximum_pcv-minimum_pcv)
    df_pc['momentum pcv away previous 2 minutes'] = (df_pc['momentum pcv away previous 2 minutes']-minimum_pcv)/(maximum_pcv-minimum_pcv)
    df_pc['pcv home'] = (df_pc['pcv home']-minimum_pcv)/(maximum_pcv-minimum_pcv)
    df_pc['pcv away'] = (df_pc['pcv away']-minimum_pcv)/(maximum_pcv-minimum_pcv)
    df_pc['momentum pc home previous 2 minutes'] = (df_pc['momentum pc home previous 2 minutes']-minimum_pc)/(maximum_pc-minimum_pc)
    df_pc['momentum pc away previous 2 minutes'] = (df_pc['momentum pc away previous 2 minutes']-minimum_pc)/(maximum_pc-minimum_pc)
    df_pc['pc home'] = (df_pc['pc home']-minimum_pc)/(maximum_pc-minimum_pc)
    df_pc['pc away'] = (df_pc['pc away']-minimum_pc)/(maximum_pc-minimum_pc)
    sh = df_pc.loc[df_pc['Half'] == 2]
    
    #open match decision dataframe
    infile = open('Decision matches dataframe.p','rb')
    df_decisions = pickle.load(infile)
    infile.close()
    
    #take only second half and above the 50th minute for each match
    df_decisions = df_decisions.loc[df_decisions['Match'] == game]
    df_decisions = df_decisions.loc[df_decisions['half'] == '2']
    df_decisions = df_decisions[['index', 'type_id', 'type_name', 'half', 'frame_id', 'minute', 'seconds', 'team_id', 'team', 'formation_change', 'home score', 'away score', 'current_formation_home', 'current_formation_away', 'Riskyness decision', 'Match']]
    df_decisions['minute'] = pd.to_numeric(df_decisions['minute'])
    df_decisions = df_decisions.loc[df_decisions['minute'] >= 50]
    
    
    deltamompchome = []
    deltamompcaway = []
    deltamompcvhome = []
    deltamompcvaway = []
    meanmompchome = []
    meanmompcaway = []
    meanmompcvhome = []
    meanmompcvaway = []
    meanmomallpchome = []
    meanmomallpcaway = []
    meanmomallpcvhome = []
    meanmomallpcvaway = []
    varmompchome = []
    varmompcaway = []
    varmompcvhome = []
    varmompcvaway = []
    index_min_below = []
    index_min_above = []
    indexa = []
    indexb = []

    #for each row in the decision dataframe: calculate the indices of the time windows corresponding to the pitch control datframe
    #and add for the home and away team the mean pitch control values 10 min before the decision is made
    #these calculations are in the basis the same as in the other file for the effect of key moments on the flow of momentum.
    #therefore, I will not comment them in detail

    for i in range(len(df_decisions)):
        minute = float(df_decisions.iloc[i]['minute'])
        second = float(df_decisions.iloc[i]['seconds'])
        try:
            index_above = sh[sh['Time'] > float((minute-45) + second/60)].iloc[0].name
        except:
            index_above = sh[sh['Time'] < float((minute-45) + second/60)].iloc[-1].name
        try:
            index_below = sh[sh['Time'] <= float((minute-45) + second/60)].iloc[-1].name
        except:
            index_below = index_above

        index_10min_above = sh.index[sh.index <= index_above +10*60*25].max()
        index_10min_below = sh.index[sh.index >= index_below -10*60*25].min()
        df_i_pc = sh.loc[index_10min_below:index_below]
        deltamompchome.append(df_i_pc.iloc[-1]['momentum pc home previous 2 minutes'] - df_i_pc.iloc[0]['momentum pc home previous 2 minutes'])
        deltamompcaway.append(df_i_pc.iloc[-1]['momentum pc away previous 2 minutes'] - df_i_pc.iloc[0]['momentum pc away previous 2 minutes'])
        deltamompcvhome.append(df_i_pc.iloc[-1]['momentum pcv home previous 2 minutes'] - df_i_pc.iloc[0]['momentum pcv home previous 2 minutes'])
        deltamompcvaway.append(df_i_pc.iloc[-1]['momentum pcv away previous 2 minutes'] - df_i_pc.iloc[0]['momentum pcv away previous 2 minutes'])
        meanmompchome.append(df_i_pc['momentum pc home previous 2 minutes'].mean())
        meanmompcaway.append(df_i_pc['momentum pc away previous 2 minutes'].mean())
        meanmompcvhome.append(df_i_pc['momentum pcv home previous 2 minutes'].mean())
        meanmompcvaway.append(df_i_pc['momentum pcv away previous 2 minutes'].mean())
        meanmomallpchome.append(df_pc.loc[0:index_below]['momentum pc home previous 2 minutes'].mean())
        meanmomallpcaway.append(df_pc.loc[0:index_below]['momentum pc away previous 2 minutes'].mean())
        meanmomallpcvhome.append(df_pc.loc[0:index_below]['momentum pcv home previous 2 minutes'].mean())
        meanmomallpcvaway.append(df_pc.loc[0:index_below]['momentum pcv away previous 2 minutes'].mean())
        varmompchome.append(sum(abs(df_i_pc['momentum pc home previous 2 minutes'].diff()[1:])))
        varmompcaway.append(sum(abs(df_i_pc['momentum pc away previous 2 minutes'].diff()[1:])))
        varmompcvhome.append(sum(abs(df_i_pc['momentum pcv home previous 2 minutes'].diff()[1:])))
        varmompcvaway.append(sum(abs(df_i_pc['momentum pcv away previous 2 minutes'].diff()[1:])))
        index_min_below.append(index_10min_below)
        index_min_above.append(index_10min_above)
        indexa.append(index_above)
        indexb.append(index_below)



    df_decisions['delta mom pc home'] = deltamompchome
    df_decisions['delta mom pc away'] = deltamompcaway
    df_decisions['delta mom pcv home'] = deltamompcvhome
    df_decisions['delta mom pcv away'] = deltamompcvaway
    df_decisions['average mom pc home'] = meanmompchome
    df_decisions['average mom pc away'] = meanmompcaway
    df_decisions['average mom pcv home'] = meanmompcvhome
    df_decisions['average mom pcv away'] = meanmompcvaway
    df_decisions['average mom all pc home'] = meanmomallpchome
    df_decisions['average mom all pc away'] = meanmomallpcaway
    df_decisions['average mom all pcv home'] = meanmomallpcvhome
    df_decisions['average mom all pcv away'] = meanmomallpcvaway
    df_decisions['var mom pc home'] = varmompchome
    df_decisions['var mom pc away'] = varmompcaway
    df_decisions['var mom pcv home'] = varmompcvhome
    df_decisions['var mom pcv away'] = varmompcvaway
    df_decisions['index 10min before'] = index_min_below
    df_decisions['index 10min after'] = index_min_above
    df_decisions['index below'] = indexb
    df_decisions['index above'] = indexa
    
    minutes_second_half = []
    
    #check if there are time windows where no decisions are made. If yes, add these time windows to the dataframe

    for i in range(len(df_decisions)):
        try: 
            if i == 0:
                if df_decisions.iloc[i]['index below'] - sh.iloc[0].name > 20*60*25:
                    dif = df_decisions.iloc[i]['index below'] - sh.iloc[0].name
                    if dif < 30*60*25:
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int(dif/2)]['Time'])
                    elif dif >= 30*60*25:
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int(dif/3)]['Time'])
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int((dif/3)*2)]['Time'])                     

            if df_decisions.iloc[i+1]['index below'] - df_decisions.iloc[i]['index below'] > 20*60*25:
                dif = df_decisions.iloc[i+1]['index below'] - df_decisions.iloc[i]['index below']
                if dif < 30*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index below'] + int(dif/2)]['Time'])
                elif dif >= 30*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index below'] + int(dif/3)]['Time'])
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index below'] + int((dif/3)*2)]['Time'])

        except:
            if sh.iloc[-1].name - df_decisions.iloc[i]['index below'] > 20*60*25:
                dif = sh.iloc[-1].name - df_decisions.iloc[i]['index below']
                if dif < 30*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index below'] + int(dif/2)]['Time'])
                elif dif >= 30*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index below'] + int(dif/3)]['Time'])
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index below'] + int((dif/3)*2)]['Time'])
    minutes_second_half = [i+45 for i in minutes_second_half]
    
    deltamompchome = []
    deltamompcaway = []
    deltamompcvhome = []
    deltamompcvaway = []
    meanmompchome = []
    meanmompcaway = []
    meanmompcvhome = []
    meanmompcvaway = []
    meanmomallpchome = []
    meanmomallpcaway = []
    meanmomallpcvhome = []
    meanmomallpcvaway = []
    varmompchome = []
    varmompcaway = []
    varmompcvhome = []
    varmompcvaway = []
    index_min_below = []
    index_min_above = []
    indexa = []
    indexb = []
    minutes = []
    seconds = []
    half = []

    #do the same for the second half
    for i in range(len(minutes_second_half)):
        minute = float(int(minutes_second_half[i]))
        second = float((minutes_second_half[i] - int(minutes_second_half[i]))*60)
        try:
            index_above = sh[sh['Time'] > float((minute-45) + second/60)].iloc[0].name
        except:
            index_above = sh[sh['Time'] < float((minute-45) + second/60)].iloc[-1].name
        try:
            index_below = sh[sh['Time'] <= float((minute-45) + second/60)].iloc[-1].name
        except:
            index_below = index_above

        index_10min_above = sh.index[sh.index <= index_above +10*60*25].max()
        index_10min_below = sh.index[sh.index >= index_below -10*60*25].min()
        df_i_pc = sh.loc[index_10min_below:index_below]
        deltamompchome.append(df_i_pc.iloc[-1]['momentum pc home previous 2 minutes'] - df_i_pc.iloc[0]['momentum pc home previous 2 minutes'])
        deltamompcaway.append(df_i_pc.iloc[-1]['momentum pc away previous 2 minutes'] - df_i_pc.iloc[0]['momentum pc away previous 2 minutes'])
        deltamompcvhome.append(df_i_pc.iloc[-1]['momentum pcv home previous 2 minutes'] - df_i_pc.iloc[0]['momentum pcv home previous 2 minutes'])
        deltamompcvaway.append(df_i_pc.iloc[-1]['momentum pcv away previous 2 minutes'] - df_i_pc.iloc[0]['momentum pcv away previous 2 minutes'])
        meanmompchome.append(df_i_pc['momentum pc home previous 2 minutes'].mean())
        meanmompcaway.append(df_i_pc['momentum pc away previous 2 minutes'].mean())
        meanmompcvhome.append(df_i_pc['momentum pcv home previous 2 minutes'].mean())
        meanmompcvaway.append(df_i_pc['momentum pcv away previous 2 minutes'].mean())
        meanmomallpchome.append(df_pc.loc[0:index_below]['momentum pc home previous 2 minutes'].mean())
        meanmomallpcaway.append(df_pc.loc[0:index_below]['momentum pc away previous 2 minutes'].mean())
        meanmomallpcvhome.append(df_pc.loc[0:index_below]['momentum pcv home previous 2 minutes'].mean())
        meanmomallpcvaway.append(df_pc.loc[0:index_below]['momentum pcv away previous 2 minutes'].mean())
        varmompchome.append(sum(abs(df_i_pc['momentum pc home previous 2 minutes'].diff()[1:])))
        varmompcaway.append(sum(abs(df_i_pc['momentum pc away previous 2 minutes'].diff()[1:])))
        varmompcvhome.append(sum(abs(df_i_pc['momentum pcv home previous 2 minutes'].diff()[1:])))
        varmompcvaway.append(sum(abs(df_i_pc['momentum pcv away previous 2 minutes'].diff()[1:])))
        index_min_below.append(index_10min_below)
        index_min_above.append(index_10min_above)
        indexa.append(index_above)
        indexb.append(index_below)
        minutes.append(minute + 45)
        seconds.append(second)
        half.append(2)
    
    index = []
    type_id = []
    type_name = []
    half = []
    frame_id = []
    team_id = []
    team = []
    formation_change = []
    current_formation_home = []
    current_formation_away = []
    riskyness_decision = []
    match_id = []
    half = []
    minute = []
    seconds = []

    #add the new time windows with the data to the original decision dataframe
    for j in range(len(minutes_second_half)):
        index.append(df_decisions.iloc[-1].name +j)
        type_id.append(0)
        type_name.append('No')
        team_id.append(0)
        team.append('Neutral')
        match_id.append(df_decisions.iloc[0]['Match'])
        formation_change.append(0)
        frame_id.append(0)
        current_formation_home.append(0)
        current_formation_away.append(0)
        riskyness_decision.append('risk_neutral')
        half.append('2')
        minute.append(int(minutes_second_half[j]))
        seconds.append((minutes_second_half[j] - int(minutes_second_half[j]))*60)
        
    data = {'index': index,
           'type_id': type_id,
           'type_name': type_name,
           'half': half,
           'frame_id': frame_id,
           'minute': minute,
           'seconds': seconds,
           'team_id': team_id,
           'team': team,
            'formation_change' : formation_change,
            'current_formation_home' : current_formation_home,
            'current_formation_away' : current_formation_away,
            'Riskyness decision' : riskyness_decision,
           'Match': match_id,
            'delta mom pc home': deltamompchome,
            'delta mom pc away': deltamompcaway,
            'delta mom pcv home': deltamompcvhome,
            'delta mom pcv away': deltamompcvaway,
            'average mom pc home': meanmompchome,
            'average mom pc away': meanmompcaway,
            'average mom pcv home': meanmompcvhome,
            'average mom pcv away': meanmompcvaway,
            'average mom all pc home': meanmomallpchome,
            'average mom all pc away': meanmomallpcaway,
            'average mom all pcv home': meanmomallpcvhome,
            'average mom all pcv away': meanmomallpcvaway,
            'var mom pc home': varmompchome,
            'var mom pc away': varmompcaway,
            'var mom pcv home': varmompcvhome,
            'var mom pcv away': varmompcvaway,
            'index 10min before': index_min_below,
            'index 10min after': index_min_above,
           'index below': indexb,
           'index above': indexa}

    df = pd.DataFrame(data)
    df_new = pd.concat([df_decisions, df])

    
    #open the shots and goals dataframe
    #make a list with who scored in which minute

    infile = open('shots and goals event dataframes.p','rb')
    df_shot = pickle.load(infile)
    infile.close()
    goals = []
    df_shots = df_shot.loc[df_shot['match_id'] == game]
    for i in range(len(df_shots)):
        if df_shots.iloc[i]['type_id'] == '16':
            goals.append([int(df_shots.iloc[i]['minute']) + int(df_shots.iloc[i]['seconds'])/60, df_shots.iloc[i]['team']])

    time_left = []
    time_previous_goal = []
    home_goals = []
    away_goals = []
    #for every row in the dataframe: add the time left, the home and away score and the time since the previous goal
    for i in range(len(df_new)):
        home = 0
        away = 0
        time_left.append(90 - (int(df_new.iloc[i]['minute']) + int(df_new.iloc[i]['seconds'])/60))
        for j in range(len(goals)):
            if (int(df_new.iloc[i]['minute']) + int(df_new.iloc[i]['seconds'])/60) >= goals[j][0] and goals[j][1] == 'Home':
                home = home + 1
            elif (int(df_new.iloc[i]['minute']) + int(df_new.iloc[i]['seconds'])/60) >= goals[j][0] and goals[j][1] == 'Away':
                away = away + 1
        home_goals.append(home)
        away_goals.append(away)
        minute_now = (int(df_new.iloc[i]['minute']) + int(df_new.iloc[i]['seconds'])/60)
        goals_new = [i[0] for i in goals if i[0] < minute_now]
        if len(goals_new) == 0:
            time_previous_goal.append(int(df_new.iloc[i]['minute']) + int(df_new.iloc[i]['seconds'])/60)
        else:
            time_previous_goal.append((int(df_new.iloc[i]['minute']) + int(df_new.iloc[i]['seconds'])/60) - goals_new[-1])


    df_new['Time left'] = time_left
    df_new['Home score'] = home_goals
    df_new['Away score'] = away_goals
    df_new['Time last goal scored'] = time_previous_goal
    
    final_df = pd.concat([final_df, df_new])

#create pickle file for all the matches that will be used as input for regression
pickle.dump(final_df, open('input PC regression Momentum on decisions.p', 'wb')) 





In [ ]:
#Calculations that will be used for regression input for the flow of momentum on decision making (xT calculations)
#These are again approximately the same as the PC and PCV models, and the xT calculations for the effect of key moments on the flow of momentum
from progressbar import ProgressBar
import pickle
import pandas as pd

#open decisions dataframe
infile = open('Decision matches dataframe.p','rb')
df_decisions = pickle.load(infile)
infile.close()

#open dataframe with xT for all matches per half minute
infile = open('all matches sliding windows xT.p','rb')
df_xT = pickle.load(infile)
infile.close()

maximum = max(df_xT['xT_home'].max(), df_xT['xT_away'].max())
minimum = min(df_xT['xT_home'].min(), df_xT['xT_away'].min())

df_xT['xT_home'] = (df_xT['xT_home']-minimum)/(maximum-minimum)
df_xT['xT_away'] = (df_xT['xT_away']-minimum)/(maximum-minimum)
df_xT['xT_prev_2min_home'] = (df_xT['xT_prev_2min_home']-minimum)/(maximum-minimum)
df_xT['xT_prev_2min_away'] = (df_xT['xT_prev_2min_away']-minimum)/(maximum-minimum)

#create an array with all matches in the xT dataframe
matches = df_xT.Match.unique()
pbar = ProgressBar()
final_df = pd.DataFrame()

#loop over all matches in the xT dataframe
for game in pbar(matches):
    #only take the second half of each match in the dataframe
    match_decisions = df_decisions.loc[df_decisions['Match'] == game]
    match_decisions = match_decisions.loc[match_decisions['half'] == '2']
    
    #find time time windows where no decision is made
    minutes_second_half = []
    for i in range(len(match_decisions)):
        try:
            if i == 0:
                if (int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60) - 45  > 20:
                    dif = (int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60) - 45
                    if dif < 30:
                        minutes_second_half.append(int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60 - dif/2)
                    elif dif >= 30:
                        minutes_second_half.append(int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60 - dif/3)
                        minutes_second_half.append(int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60 - 2*dif/3)


            if (int(match_decisions.iloc[i+1]['minute'])+int(match_decisions.iloc[i+1]['seconds'])/60) - (int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60) > 20:
                dif = (int(match_decisions.iloc[i+1]['minute'])+int(match_decisions.iloc[i+1]['seconds'])/60) - (int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60)
                if dif < 30:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 30:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + dif/3)
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + 2*dif/3)

        except:
            if 91 - (int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60) > 20:
                dif = 91 - (int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60)
                if dif < 30:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 30:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + dif/3)
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + 2*dif/3)
        

    match_decisions = match_decisions[['index', 'type_id', 'type_name', 'half', 'frame_id', 'minute', 'seconds', 'team_id', 'team', 'formation_change', 'current_formation_home', 'current_formation_away', 'Riskyness decision', 'Match']]

    index = []
    type_id = []
    type_name = []
    half = []
    frame_id = []
    team_id = []
    team = []
    formation_change = []
    current_formation_home = []
    current_formation_away = []
    riskyness_decision = []
    match_id = []
    half = []
    minute = []
    seconds = []

    #add these new time windows to the original dataframe
    for j in range(len(minutes_second_half)):
        index.append(match_decisions.iloc[-1].name +j)
        type_id.append(0)
        type_name.append('No')
        team_id.append(0)
        team.append('risk_neutral')
        match_id.append(match_decisions.iloc[0]['Match'])
        formation_change.append(0)
        frame_id.append(0)
        current_formation_home.append(0)
        current_formation_away.append(0)
        riskyness_decision.append('neutral')
        half.append('2')
        minute.append(int(minutes_second_half[j]))
        seconds.append((minutes_second_half[j] - int(minutes_second_half[j]))*60)


    data = {'index': index,
           'type_id': type_id,
           'type_name': type_name,
           'half': half,
           'frame_id': frame_id,
           'minute': minute,
           'seconds': seconds,
           'team_id': team_id,
           'team': team,
            'formation_change' : formation_change,
            'current_formation_home' : current_formation_home,
            'current_formation_away' : current_formation_away,
            'Riskyness decision' : riskyness_decision,
           'Match': match_id}

    df_new = pd.DataFrame(data)
    df = pd.concat([match_decisions, df_new])
    
    #open the shots and goals dataframe
    infile = open('shots and goals event dataframes.p','rb')
    df_shot = pickle.load(infile)
    infile.close()
    goals = []
    df_shots = df_shot.loc[df_shot['match_id'] == game]
    
    #create a list with each goal in the match and who scored
    for i in range(len(df_shots)):
        if df_shots.iloc[i]['type_id'] == '16':
            goals.append([int(df_shots.iloc[i]['minute']) + int(df_shots.iloc[i]['seconds'])/60, df_shots.iloc[i]['team']])

    time_left = []
    home_goals = []
    away_goals = []
    time_previous_goal = []
    
    #for each row in the match dataframe: add time left, the home and away score, and the time since the previous goal
    for i in range(len(df)):
        home = 0
        away = 0
        time_left.append(90 - (int(df.iloc[i]['minute']) + int(df.iloc[i]['seconds'])/60))
        for j in range(len(goals)):
            if (int(df.iloc[i]['minute']) + int(df.iloc[i]['seconds'])/60) >= goals[j][0] and goals[j][1] == 'Home':
                home = home + 1
            elif (int(df.iloc[i]['minute']) + int(df.iloc[i]['seconds'])/60) >= goals[j][0] and goals[j][1] == 'Away':
                away = away + 1
        home_goals.append(home)
        away_goals.append(away)
        minute_now = (int(df.iloc[i]['minute']) + int(df.iloc[i]['seconds'])/60)
        goals_new = [i[0] for i in goals if i[0] < minute_now]
        if len(goals_new) == 0:
            time_previous_goal.append(int(df.iloc[i]['minute']) + int(df.iloc[i]['seconds'])/60)
        else:
            time_previous_goal.append((int(df.iloc[i]['minute']) + int(df.iloc[i]['seconds'])/60) - goals_new[-1])

    df['Time left'] = time_left
    df['Home score'] = home_goals
    df['Away score'] = away_goals
    df['Time last goal scored'] = time_previous_goal
    
    #create the final dataframe but still without the xT calculations
    final_df = pd.concat([final_df, df])


In [ ]:
pbar = ProgressBar()
#add the xT values to the dataframe
df_final = pd.DataFrame()
#for each match in the dataframe: take only the data from the 50th minute
for game in pbar(matches):
    df_decisions = final_df.loc[final_df['Match'] == game]
    df_decisions['minute'] = pd.to_numeric(df_decisions['minute'])
    df_decisions = df_decisions.loc[df_decisions['minute'] >= 50]
    df_match_xT = df_xT.loc[df_xT['Match'] == game]
    df_xT_sh = df_match_xT.loc[df_match_xT['Half'] == 2]
    
    deltamomxthome = []
    deltamomxtaway = []
    meanmomxthome = []
    meanmomxtaway = []
    meanmomallhome = []
    meanmomallaway = []
    varmomhome = []
    varmomaway = []
    
    
    
    #add the xT values to the dataframe again making sure to weigh them properly 
    for i in range(len(df_decisions)):
        time = int(df_decisions.iloc[i]['minute']) + int(df_decisions.iloc[i]['seconds'])/60
        df_all = df_match_xT.loc[df_match_xT['Minute'] <= time]
        df_now  =  df_xT_sh.loc[df_xT_sh['Minute'] <= time]
        df_10min = df_now.iloc[-min(21, len(df_now)):-1]
        deltamomxthome.append(df_10min.iloc[-1]['xT_prev_2min_home'] - df_10min.iloc[0]['xT_prev_2min_home'])
        deltamomxtaway.append(df_10min.iloc[-1]['xT_prev_2min_away'] - df_10min.iloc[0]['xT_prev_2min_away'])
        meanmomxthome.append(df_10min['xT_prev_2min_home'].mean())
        meanmomxtaway.append(df_10min['xT_prev_2min_away'].mean())
        meanmomallhome.append(df_all['xT_prev_2min_home'].mean())
        meanmomallaway.append(df_all['xT_prev_2min_away'].mean())
        varmomhome.append(sum(abs(df_10min['xT_prev_2min_home'].diff()[1:])))
        varmomaway.append(sum(abs(df_10min['xT_prev_2min_away'].diff()[1:])))
        
        

            
    df_decisions['delta momentum home previous 10 min'] = deltamomxthome
    df_decisions['delta momentum away previous 10 min'] = deltamomxtaway
    df_decisions['mean momentum home previous 10 min'] = meanmomxthome
    df_decisions['mean momentum away previous 10 min'] = meanmomxtaway
    df_decisions['mean momentum home all minutes'] = meanmomallhome
    df_decisions['mean momentum away all minutes'] = meanmomallaway
    df_decisions['var momentum home previous 10 min'] = varmomhome
    df_decisions['var momentum away previous 10 min'] = varmomaway
    
    
    #for each match add the complete dataframe to the final dataframe
    df_final = pd.concat([df_decisions, df_final])

#export the complete dataframe for all matches to pickle
pickle.dump(df_final, open('input xT regression Momentum on decisions.p', 'wb')) 